<a href="https://colab.research.google.com/github/EdisonWang00/production-planning/blob/main/production_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import io
import matplotlib.pyplot as plt
import re

data = files.upload()

Saving demand_forecasts.csv to demand_forecasts.csv


In [5]:
data=pd.read_csv("demand_forecasts.csv",index_col=False)
data

,period;forecast
0,1;200
1,2;150
2,3;100
3,4;50
4,5;50
5,6;100
6,7;150
7,8;200
8,9;200
9,10;250


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   period    12 non-null     object
 1   forecast  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [6]:
# Split the column into two columns based on ';'
data[['period', 'forecast']] = data['period;forecast'].str.split(';', 1, expand=True)

# Drop the original column if needed
data = data.drop('period;forecast', axis=1)

<ipython-input-6-e2e59fe9e63d>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  data[['period', 'forecast']] = data['period;forecast'].str.split(';', 1, expand=True)


In [15]:
data['period'] = pd.to_numeric(data['period'], downcast='integer')
data['forecast'] = pd.to_numeric(data['forecast'], downcast='integer')


In [16]:
# Create columns
data_calc = data.copy()

for i in data_calc['period'].unique():
    data_calc['Order {}'.format(i)] = 0


# costs
set_up = 500
holding = 1

# Order 1
order = 1
for index, row in data_calc.iterrows():
    current_month = data_calc.loc[index,'period']
    cost = 0
    # 1 set up
    cost += set_up
    if current_month > 1:
        for t in range(1, current_month+1):
            cost += (t-1) * data_calc.loc[t-1,'forecast'] * holding
    data_calc.loc[index,'Order {}'.format(order)] = cost

# Order 2
for order in range(2, 13):
    for index, row in data_calc.iterrows():
        current_month = data_calc.loc[index,'period']
        if current_month >= order:
            cost = 0

            # Best option best Period 1
            values = list(data_calc.loc[order-2,['Order {}'.format(i) for i in range(1, order+1)]].values)
            best = min([i for i in values if i >0])

            # Add
            cost += best + set_up
            for t in range(order, current_month+1):
                cost += (t-order) * data_calc.loc[t-1,'forecast'] * holding
            data_calc.loc[index,'Order {}'.format(order)] = cost

data_calc = data_calc.set_index('period').drop(['forecast'], axis = 1).T
data_calc

period,1,2,3,4,5,6,7,8,9,10,11,12
Order 1,500,650,850,1000,1200,1700,2600,4000,5600,7850,10850,13600
Order 2,0,1000,1100,1200,1350,1750,2500,3700,5100,7100,9800,12300
Order 3,0,0,1150,1200,1300,1600,2200,3200,4400,6150,8550,10800
Order 4,0,0,0,1350,1400,1600,2050,2850,3850,5350,7450,9450
Order 5,0,0,0,0,1500,1600,1900,2500,3300,4550,6350,8100
Order 6,0,0,0,0,0,1700,1850,2250,2850,3850,5350,6850
Order 7,0,0,0,0,0,0,2100,2300,2700,3450,4650,5900
Order 8,0,0,0,0,0,0,0,2350,2550,3050,3950,4950
Order 9,0,0,0,0,0,0,0,0,2750,3000,3600,4350
Order 10,0,0,0,0,0,0,0,0,0,3050,3350,3850


In [17]:
costs, initials, nexts, quantities = [], [], [], []
i = 12
while i>1:
    # Order with the minimum cost
    initial_step = i
    next_step = data_calc[data_calc[i]>0][i].idxmin()
    cost = data_calc[data_calc[i]>0][i].min()
    # Next Step
    next_id = int(next_step.replace('Order ',''))
    i = next_id - 1
    # Quantity
    quantity = data[data['period'].isin(range(next_id, initial_step+1))]['forecast'].sum()

    costs.append(cost)
    initials.append(initial_step)
    nexts.append(next_id)
    quantities.append(quantity)

df_results = pd.DataFrame({'backward':range(1, len(initials)+1),
                           'initial':initials,
                           'nexts':nexts,
                           'cost':costs,
                           'quantity':quantities}).set_index('backward')
print("Total Cost: {:,}$".format(df_results.cost.sum()))
df_results

Total Cost: 10,200$


,initial,nexts,cost,quantity
backward,,,,
1,12,11,3750,550
2,10,9,3000,450
3,8,6,2250,450
4,5,1,1200,550


In [18]:
# Holding unit cost per month
hold_cost = 1
# Set Up Cost
setup_cost = 500

# Final Results
results_final = data.copy()

# Production
month_prod = df_results['nexts'].values
prod_dict = dict(zip(month_prod, df_results.quantity.values))

# Values
results_final['production'] = results_final['period'].apply(lambda t: prod_dict[t] if t in month_prod else 0)

# Inventory On Hand
results_final['IOH'] = (results_final['production'] - results_final['forecast']).cumsum()

# Holding Cost
results_final['Holding Cost'] = (results_final['IOH'] * hold_cost)

# Set Up Cost
results_final['Set-Up Costs'] = results_final['production'].apply(lambda t: setup_cost if t > 0 else 0)

# Total Cost
results_final['Total Cost'] = results_final[['Holding Cost', 'Set-Up Costs']].sum(axis = 1)

results_final

,period,forecast,production,IOH,Holding Cost,Set-Up Costs,Total Cost
0,1,200,550,350,350,500,850
1,2,150,0,200,200,0,200
2,3,100,0,100,100,0,100
3,4,50,0,50,50,0,50
4,5,50,0,0,0,0,0
5,6,100,450,350,350,500,850
6,7,150,0,200,200,0,200
7,8,200,0,0,0,0,0
8,9,200,450,250,250,500,750
9,10,250,0,0,0,0,0
